<a href="https://colab.research.google.com/github/BDonadelli/Codigos-em-financas/blob/main/momentum_simples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ticker="ambp3"
import yfinance as yf
data = yf.download(ticker+'.SA',
                    # start="2017-01-01",end="2017-04-30", 
                    period='max',
                    auto_adjust=True)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2020-07-13,24.931172,25.762210,24.808702,25.630993,12447334
2020-07-14,25.622244,28.998886,25.622244,27.555504,3067762
2020-07-15,28.421535,28.858923,25.630992,26.986898,1855107
2020-07-16,26.593249,26.768205,25.849688,25.980904,1802310
2020-07-17,25.998398,26.724465,25.456036,26.112120,1054043
...,...,...,...,...,...
2024-09-30,134.679993,149.740005,134.500000,137.000000,205000
2024-10-01,124.989998,135.000000,107.500000,135.000000,276200
2024-10-02,138.000000,147.429993,136.000000,139.869995,444300


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:

def strategy(returns, window=1, short=True):
    '''
        estratégia:
            - compra se o retorno na janela é >0
            - venda  se o retorno na janela é <0, opcionalmente 
    '''

    if short:
        position = returns.rolling(window).mean().map(lambda x: -1 if x <= 0 else 1)
    else:
        position = returns.rolling(window).mean().map(lambda x: 0 if x <= 0 else 1)
    performance = position.shift(1) * returns
    return performance


In [12]:
## log returns
returns = np.log(data['Close'])-np.log(data['Close'].shift(1))
returns = returns.dropna()

performance = strategy(returns,5 , True).dropna()
performance.tail()


Date
2024-09-30    0.018417
2024-10-01   -0.014706
2024-10-02    0.035439
2024-10-03    0.063233
2024-10-04   -0.098672
Name: Close, dtype: float64

In [13]:
n_years = (performance.index.max() - performance.index.min()).days / 252
perf_cum = np.exp(performance.cumsum())
tot = perf_cum.iloc[-1] - 1
ann = perf_cum.iloc[-1] ** (1 / n_years) - 1
vol = performance.std() * np.sqrt(252)
rfr = 0.1075
sharpe = (ann - rfr) / vol
print("Retorno da estratégia")
print(f"\t{tot*100:.2f}% retorno total" +
      f"\n\t{ann*100:.2f}% anual" +
      f"\n\t{sharpe:.2f} Sharpe Ratio")


Retorno da estratégia
	1310.22% retorno total
	54.11% anual
	0.70 Sharpe Ratio


In [14]:
ret = np.exp(returns.cumsum())
b_tot = ret.iloc[-1] - 1
b_ann = ret.iloc[-1] ** (1 / n_years) - 1
b_vol = returns.std() * np.sqrt(252)
b_sharpe = (b_ann - rfr) / b_vol
print(f"Buy-and-Hold:" +
      f"\n\t{b_tot*100:.2f}%  retorno total" +
      f"\n\t{b_ann*100:.2f}% anual" +
      f"\n\t{b_sharpe:.2f} Sharpe Ratio")

Buy-and-Hold:
	426.71%  retorno total
	31.20% anual
	0.33 Sharpe Ratio


In [15]:
# simula estratégia em janelas de  3, 5, ...  dias
periods = [3, 5, 7, 11, 13, 17, 23, 29, 31, 37, 43 , 47, 53, 59,61, 67,71,87]
# periods = [43,59]

In [16]:
perf_dict = {'tot_ret': {'buy_and_hold': (np.exp(returns.sum()) - 1)}}
perf_dict['ann_ret'] = {'buy_and_hold': b_ann}
perf_dict['sharpe'] = {'buy_and_hold': b_sharpe}
for p in periods:
    log_perf = strategy(returns, window=p )
    perf = np.exp(log_perf.cumsum())
    perf_dict['tot_ret'][p] = (perf.iloc[-1] - 1)
    ann = (perf.iloc[-1] ** (1/n_years) - 1)
    perf_dict['ann_ret'][p] = ann
    vol = log_perf.std() * np.sqrt(252)
    perf_dict['sharpe'][p] = (ann - rfr) / vol

In [17]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=returns.index, 
    y=(np.exp(returns.cumsum()) - 1) * 100, 
    mode='lines', 
    name=ticker
))

for p in periods:
    log_perf = strategy(returns, window=p)
    perf = np.exp(log_perf.cumsum())
    fig.add_trace(go.Scatter(
        x=returns.index, 
        y=(perf - 1) * 100, 
        mode='lines', 
        name=f'{p}-Day Mean'
    ))

fig.update_layout(
    title='Retorno Acumulado',
    xaxis_title='Data',
    yaxis_title='Retorno (%)',
    legend_title='Estratégias',
    showlegend=True
)

fig.show()


In [18]:
fig_ann_ret = go.Figure()

# Adicionando as barras de Retorno Anualizado
fig_ann_ret.add_trace(go.Bar(
    x=[f'{k}-Day Mean' if isinstance(k, int) else ticker for k in perf_dict['ann_ret'].keys()],
    y=[v * 100 for v in perf_dict['ann_ret'].values()],
    name='Retorno Anualizado'
))

# Ajustando o layout do gráfico
fig_ann_ret.update_layout(
    title='Retorno Anualizado por Estratégia',
    xaxis_title='Estratégia',
    yaxis_title='Retorno (%)',
    xaxis_tickangle=45,
    showlegend=False
)

# Exibindo o gráfico
fig_ann_ret.show()


In [19]:
fig_sharpe = go.Figure()

# Adicionando as barras de Sharpe Ratio
fig_sharpe.add_trace(go.Bar(
    x=[f'{k}-Day Mean' if isinstance(k, int) else ticker for k in perf_dict['sharpe'].keys()],
    y=[v for v in perf_dict['sharpe'].values()],
    name='Sharpe Ratio'
))

# Ajustando o layout do gráfico
fig_sharpe.update_layout(
    title='Sharpe Ratio por Estratégia',
    xaxis_title='Estratégia',
    yaxis_title='Sharpe Ratio',
    xaxis_tickangle=45,
    showlegend=False
)

# Exibindo o gráfico
fig_sharpe.show()
